In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

import keras  
from keras import models
from keras import optimizers
from keras.layers import Conv1D,Flatten,AveragePooling1D,Dense,Embedding,Reshape,Input,Concatenate
from keras.models import model_from_json

import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/train.csv') #load the  training dataset
test=pd.read_csv('/gdrive/My Drive/Colab Notebooks/megaproject2/test.csv') #load the test dataset


Normalization

In [ ]:
'''normalization for train'''
cols_to_norm = ['Crop_Year']
meanyear = train[cols_to_norm].mean()
stdyear = train[cols_to_norm].std()
train[cols_to_norm] -= meanyear
train[cols_to_norm] /=stdyear


cols_to_norm = ['Area']
meanarea = train[cols_to_norm].mean()
stdarea = train[cols_to_norm].std()
train[cols_to_norm] -= meanarea
train[cols_to_norm] /=stdarea

cols_to_norm = ['Production']
meanprod = train[cols_to_norm].mean()
stdprod = train[cols_to_norm].std()
train[cols_to_norm] -= meanprod
train[cols_to_norm] /=stdprod

'''normalization option for test'''
cols_to_norm = ['Crop_Year']
test[cols_to_norm] -= meanyear
test[cols_to_norm] /=stdyear


cols_to_norm = ['Area']
test[cols_to_norm] -= meanarea
test[cols_to_norm] /=stdarea

cols_to_norm = ['Production']
test[cols_to_norm] -= meanprod
test[cols_to_norm] /=stdprod

In [ ]:
train=train.astype(dtype='float32')
test=test.astype(dtype='float32')

classify

In [ ]:
district=train['District_Name'].values
year=train['Crop_Year'].values
season=train['Season'].values
crop=train['Crop'].values
area=train['Area'].values

district1=test['District_Name'].values
year1=test['Crop_Year'].values
season1=test['Season'].values
crop1=test['Crop'].values
area1=test['Area'].values

In [ ]:
y_train=train['Production'].values
y_test=test['Production'].values

In [ ]:
year_input = Input(shape=(1,),dtype='float32',name='year_input') #input layer for year data
year_output=Reshape(target_shape=(1,))(year_input)

area_input = Input(shape=(1,),dtype='float32',name='area_input') #input layer for area
area_output=Reshape(target_shape=(1,))(area_input)

district_input=Input(shape=(1,),dtype='float32',name='district_input') #input layer for district
no_of_district  = len(np.unique(district))
district_embed_size = min(np.ceil((no_of_district)/2), 50 )
district_embed_size = int(district_embed_size)
x = Embedding(output_dim=district_embed_size, input_dim=no_of_district, input_length=1)(district_input) #embedding layer for district_input
district_output=Reshape((district_embed_size,))(x)

season_input=Input(shape=(1,),dtype='float32',name='season_input') #input layer for season
no_of_season  = len(np.unique(season))
season_embed_size = min(np.ceil((no_of_season)/1), 50 )
season_embed_size = int(season_embed_size)
x = Embedding(output_dim=season_embed_size, input_dim=no_of_season, input_length=1)(season_input) #embedding layer for season_input
season_output=Reshape((season_embed_size,))(x)

crop_input=Input(shape=(1,),dtype='float32',name='crop_input') #input layer for crop
no_of_crop  = len(np.unique(crop))
crop_embed_size = min(np.ceil((no_of_crop)/2), 50 )
crop_embed_size = int(crop_embed_size)
x = Embedding(output_dim=crop_embed_size, input_dim=no_of_crop, input_length=1)(crop_input)  #embedding layer for crop_input
crop_output=Reshape((crop_embed_size,))(x)

combined = keras.layers.concatenate([district_output,year_output,season_output,crop_output,area_output]) #merge all ouput layers
total_size=district_embed_size+season_embed_size+crop_embed_size+1+1            # total shape of model after embedding and merging
reshaped=Reshape((total_size,1))(combined)                                      #Reshape in (none,total_size,1)

x=Conv1D(32, 1,strides=1,activation='relu', padding='same')(reshaped)
x=AveragePooling1D(1)(x)
x=Flatten()(x)
x=Dense(1124,activation='relu')(x)
x=Dense(1024,activation='relu')(x)
x=Dense(764,activation='relu')(x)
x=Dense(512,activation='relu')(x)
x=Dense(512,activation='relu')(x)
x=Dense(252,activation='relu')(x)
final_output=Dense(1,name='final_ouput')(x)                                     #final output

model =models.Model(inputs=[district_input,year_input,season_input,crop_input,area_input], outputs=[final_output])

In [ ]:
optimizers.RMSprop(lr=0.0001)
model.compile(optimizer='RMSprop', loss='mse', metrics=['mape'])

In [ ]:
model.fit([district,year,season,crop,area],y_train,batch_size=10715,epochs=25)  #train the nodel   

**model accuracy:-**
      mean absolute percetage error = 29.31

In [ ]:
loss,mape=model.evaluate([district1,year1,season1,crop1,area1],y_test)
mape

In [ ]:
value=model.predict([district,year,season,crop,area])
value *= stdprod
value += meanprod
value2=value[np.where(value<0)]
print(value2.shape)
value=model.predict([district1,year1,season1,crop1,area1])
value *= stdprod
value += meanprod
value2=value[np.where(value<0)]
print(value2.shape)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/gdrive/My Drive/Colab Notebooks/megaproject2/Saved_model/embedded_model4.json", "w") as json_file: #remove comment if need save model
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/gdrive/My Drive/Colab Notebooks/megaproject2/Saved_model/embedded_model4.h5")
print("model is saved to disk")
 



In [ ]:
# later...
 
# load json and create model
json_file = open('/gdrive/My Drive/Colab Notebooks/megaproject/Saved_model/embedded_model4.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/gdrive/My Drive/Colab Notebooks/megaproject/Saved_model/embedded_model4.h5")
print("Loaded model from disk")


In [ ]:
loaded_model.compile(optimizer='RMSprop', loss='mse', metrics=['mape'])


In [ ]:
loaded_model.fit([district,year,season,crop,area],y_train,batch_size=10715,epochs=1)

In [ ]:
mse, mape=loaded_model.evaluate([district1,year1,season1,crop1,area1],y_test)
print("mape=",mape)

In [ ]:
value=loaded_model.predict([district,year,season,crop,area])
value *= stdprod
value += meanprod
value2=value[np.where(value<0)]
print(value2.shape)
value=loaded_model.predict([district1,year1,season1,crop1,area1])
value *= stdprod
value += meanprod
value2=value[np.where(value<0)]
print(value2.shape)